## Objective: Final Prediction Computation

This notebook generates the final model predictions and formats them for submission on Codabench. 

The evaluation dataset comprises data from 39 stations included in the training set and 13 stations exclusive to the evaluation set.

<img src="../images/notebook-4.png" alt="Experiment Diagram" style="width:75%;" style="text-align:center;" />


### 1. Imports
Starts by importing the necessary libraries, configuring environment paths, and loading custom utility functions.


In [1]:
import sys
import pandas as pd
import os
import zipfile

import joblib
import pandas as pd

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..','..','..')))

from src.utils.model import load_models_auto
from src.utils.analysis import create_predict_function, create_quantile_function
from src.utils.model import load_models_auto



Defines constants :
* *DATASET_DIR* must be the directory where you unzip the *zenodo* dataset.
* *EVAL_DIR* will be used to store inference / evaluation data it must be the same as the one defined in *01 Training > 01 - Modelisation* 
* *FINAL_MODEL* will be used to store inference / evaluation data

FINAL_MODEL describe the model that will be loaded if you use auto-loading

In [2]:
ALPHA = 0.1
NUMBER_OF_WEEK = 4
USE_AUTO_SCAN = True  # Toggle this to switch between the loading of the last model of the manual load of a specific model
FINAL_MODEL = "qrf_bagging" 
DATASET_SPEC = "full_pca"

EVAL_DIR = "../../../data/evaluation/"
MODEL_DIR = f"../../../models/{DATASET_SPEC}/"

PREDS_DIR = f"{EVAL_DIR}{DATASET_SPEC}/{FINAL_MODEL}/"
os.makedirs(PREDS_DIR, exist_ok=True)

### 2. Data and models Loading

Loading of the inference dataset.

In [3]:
# load the dataset
inference_data = pd.read_csv(f"{EVAL_DIR}dataset_{DATASET_SPEC}.csv")
inference_data = inference_data.set_index("ObsDate")


Loading of the final models.

In [4]:
# Load models based on conditions
final_models = []
if FINAL_MODEL == "mapie":
    if USE_AUTO_SCAN:
        final_models = load_models_auto("mapie_quantile", f"{MODEL_DIR}final/")
    else:
        final_models.append(joblib.load(f"{MODEL_DIR}final/mapie_quantile_2025-01-17_15-15-04_week0.pkl"))
        final_models.append(joblib.load(f"{MODEL_DIR}final/mapie_quantile_2025-01-17_15-15-11_week1.pkl"))
        final_models.append(joblib.load(f"{MODEL_DIR}final/mapie_quantile_2025-01-17_15-15-17_week2.pkl"))
        final_models.append(joblib.load(f"{MODEL_DIR}final/mapie_quantile_2025-01-17_15-15-17_week3.pkl"))
elif FINAL_MODEL == "qrf":

    if USE_AUTO_SCAN:
        final_models = load_models_auto("qrf_quantile", f"{MODEL_DIR}final/")
    else:
        final_models.append(joblib.load(f"{MODEL_DIR}final/qrf_quantile_2025-01-17_15-15-04_week0.pkl"))
        final_models.append(joblib.load(f"{MODEL_DIR}final/qrf_quantile_2025-01-17_15-15-11_week1.pkl"))
        final_models.append(joblib.load(f"{MODEL_DIR}final/qrf_quantile_2025-01-17_15-15-17_week2.pkl"))
        final_models.append(joblib.load(f"{MODEL_DIR}final/qrf_quantile_2025-01-17_15-15-17_week3.pkl"))
elif FINAL_MODEL == "qrf_bagging":

    if USE_AUTO_SCAN:
        final_models = load_models_auto("qrf_bagging_quantile", f"{MODEL_DIR}final/")
    else:
        final_models.append(joblib.load(f"{MODEL_DIR}final/qrf_quantile_2025-01-17_15-15-04_week0.pkl"))
        final_models.append(joblib.load(f"{MODEL_DIR}final/qrf_quantile_2025-01-17_15-15-11_week1.pkl"))
        final_models.append(joblib.load(f"{MODEL_DIR}final/qrf_quantile_2025-01-17_15-15-17_week2.pkl"))
        final_models.append(joblib.load(f"{MODEL_DIR}final/qrf_quantile_2025-01-17_15-15-17_week3.pkl"))
elif FINAL_MODEL == "lgbm":

    if USE_AUTO_SCAN:
        models_low = load_models_auto("lgbm_quantile_q0.05", f"{MODEL_DIR}final/")
        models_med = load_models_auto("lgbm_quantile_q0.5", f"{MODEL_DIR}final/")
        models_high = load_models_auto("lgbm_quantile_q0.95", f"{MODEL_DIR}final/")
        final_models = [[] for _ in range(NUMBER_OF_WEEK)]
        final_models[0] = [models_low[0], models_med[0], models_high[0]]
        final_models[1] = [models_low[1], models_med[1], models_high[1]]
        final_models[2] = [models_low[2], models_med[2], models_high[2]]
        final_models[3] = [models_low[3], models_med[3], models_high[3]]
    else:
        final_models.append(joblib.load(f"{MODEL_DIR}final/qrf_quantile_2025-01-17_15-15-04_week0.pkl"))
        final_models.append(joblib.load(f"{MODEL_DIR}final/qrf_quantile_2025-01-17_15-15-11_week1.pkl"))
        final_models.append(joblib.load(f"{MODEL_DIR}final/qrf_quantile_2025-01-17_15-15-17_week2.pkl"))
        final_models.append(joblib.load(f"{MODEL_DIR}final/qrf_quantile_2025-01-17_15-15-17_week3.pkl"))
elif FINAL_MODEL == "ebm_ensemble":
    print("Loading EBM Ensemble")
    if USE_AUTO_SCAN:
        final_models = load_models_auto("ebm_ensemble", f"{MODEL_DIR}final/")
    else:
        final_models.append(joblib.load(f"{MODEL_DIR}final/ebm_ensemble_2025-01-17_15-15-04_week0.pkl"))
        final_models.append(joblib.load(f"{MODEL_DIR}final/ebm_ensemble_2025-01-17_15-15-11_week1.pkl"))
        final_models.append(joblib.load(f"{MODEL_DIR}final/ebm_ensemble_2025-01-17_15-15-17_week2.pkl"))
        final_models.append(joblib.load(f"{MODEL_DIR}final/ebm_ensemble_2025-01-17_15-15-17_week3.pkl"))
elif FINAL_MODEL == "deep_ensemble":
    if USE_AUTO_SCAN:
        final_models = load_models_auto("deep_ensemble", f"{MODEL_DIR}final/")
    else:
        final_models.append(joblib.load(f"{MODEL_DIR}final/deep_ensemble_2025-01-17_15-15-04_week0.pkl"))
        final_models.append(joblib.load(f"{MODEL_DIR}final/deep_ensemble_2025-01-17_15-15-11_week1.pkl"))
        final_models.append(joblib.load(f"{MODEL_DIR}final/deep_ensemble_2025-01-17_15-15-17_week2.pkl"))
        final_models.append(joblib.load(f"{MODEL_DIR}final/deep_ensemble_2025-01-17_15-15-17_week3.pkl"))


In [5]:
final_models

[BaggingRegressor(estimator=RandomForestQuantileRegressor(max_depth=30,
                                                          max_features=None,
                                                          min_samples_leaf=9,
                                                          n_estimators=45),
                  random_state=0),
 BaggingRegressor(estimator=RandomForestQuantileRegressor(max_depth=30,
                                                          max_features=None,
                                                          min_samples_leaf=9,
                                                          n_estimators=45),
                  random_state=0),
 BaggingRegressor(estimator=RandomForestQuantileRegressor(max_depth=30,
                                                          max_features=None,
                                                          min_samples_leaf=9,
                                                          n_estimators=45),
                  ran

### 3. Predictions computation

Evaluation data include a spatio-temporal split and a temporal only split.

<img src="../images/eval.png" alt="Experiment Diagram" style="width:50%;" />


In [ ]:

from typing import Any, Callable, List
import numpy as np

def create_quantile_function(
    models: List[Any], i: int, model: str, alpha: float = 0.1
) -> Callable:
    """
    Creates a quantile prediction function based on the specified model type.

    Parameters:
        model_list (List[Any]): A list of trained models.
        i (int): The index of the model to use from the list.
        model (str): The type of model, either 'mapie' or 'qrf'.
        alpha (float): The confidence level for the quantile prediction.

    Returns:
        Callable: A function that takes input data X
        and returns quantile predictions.
    """

    def predict_quantile(X):
        if model == "mapie":
            return models[i].predict(X)[1]
        if model == "lgbm":
            return np.stack([models[i][0].predict(X), models[i][2].predict(X)], axis=1)
        elif model == "qrf":
            return models[i].predict(X, quantiles=[alpha / 2, 1 - alpha / 2])
        elif model == "qrf_bagging":
            return np.mean(np.stack(
                [
                    [
                        est.predict(X, quantiles=[alpha / 2])
                        for est in models[i].estimators_
                    ],
                    [
                        est.predict(X, quantiles=[ 1 - alpha / 2])
                        for est in models[i].estimators_
                    ]
                ],
                axis=1,
            ), axis=0)
        elif model == "deep_ensemble":
            y_pred_deep = []
            for m in models[i]:
                y_pred_deep.append(m.predict(X, verbose=0))
            y_pred_deep = np.array(y_pred_deep)
            y_pred_deep = np.quantile(y_pred_deep, [alpha / 2, 1 - alpha / 2], axis=0)
            return y_pred_deep.T[0, :, :]
        raise ValueError(f"Unsupported model type: {model}")

    return predict_quantile


In [17]:
predictions = inference_data[['station_code']].copy()
y_pred_test_quantile = {}
y_pred_test = {}
X_test = inference_data.drop(columns=['station_code'])
for i in range(NUMBER_OF_WEEK):
    
    if FINAL_MODEL == "qrf":
        # reorder the columns
        X_test = X_test[final_models[0].feature_names_in_]
    print(FINAL_MODEL)
    predict_adjusted = create_predict_function(final_models, i, FINAL_MODEL)
    quantile_adjusted = create_quantile_function(final_models, i, FINAL_MODEL, ALPHA)
    
    y_pred_test[i] = predict_adjusted(X_test)
    y_pred_test_quantile[i] = quantile_adjusted(X_test)

for i in range(NUMBER_OF_WEEK):
    predictions[f"week_{i}_pred"] = y_pred_test[i]
    predictions[f"week_{i}_sup"] = y_pred_test_quantile[i][:,1]
    predictions[f"week_{i}_inf"] = y_pred_test_quantile[i][:,0]


qrf_bagging
qrf_bagging


c:\Users\chapu\E.Lucas\Perso\Github\WaterScarcity.Hackathon\.venv\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestQuantileRegressor was fitted without feature names
  warnings.warn(
c:\Users\chapu\E.Lucas\Perso\Github\WaterScarcity.Hackathon\.venv\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestQuantileRegressor was fitted without feature names
  warnings.warn(
c:\Users\chapu\E.Lucas\Perso\Github\WaterScarcity.Hackathon\.venv\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestQuantileRegressor was fitted without feature names
  warnings.warn(
c:\Users\chapu\E.Lucas\Perso\Github\WaterScarcity.Hackathon\.venv\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestQuantileRegressor was fitted without feature names
  warnings.warn(
c:\Users\chapu\E.Lucas\Perso\Github\WaterScarcity.Hackathon\.venv\Lib\site-packages\sklearn\base.py:486: Use

qrf_bagging
qrf_bagging


c:\Users\chapu\E.Lucas\Perso\Github\WaterScarcity.Hackathon\.venv\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestQuantileRegressor was fitted without feature names
  warnings.warn(
c:\Users\chapu\E.Lucas\Perso\Github\WaterScarcity.Hackathon\.venv\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestQuantileRegressor was fitted without feature names
  warnings.warn(
c:\Users\chapu\E.Lucas\Perso\Github\WaterScarcity.Hackathon\.venv\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestQuantileRegressor was fitted without feature names
  warnings.warn(
c:\Users\chapu\E.Lucas\Perso\Github\WaterScarcity.Hackathon\.venv\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestQuantileRegressor was fitted without feature names
  warnings.warn(
c:\Users\chapu\E.Lucas\Perso\Github\WaterScarcity.Hackathon\.venv\Lib\site-packages\sklearn\base.py:486: Use

ValueError: Length of values (10) does not match length of index (1352)

In [23]:
for i in range(NUMBER_OF_WEEK):
    print( np.mean(y_pred_test_quantile[i], axis=0).shape)
    predictions[f"week_{i}_sup"] = y_pred_test_quantile[i][:,1]
    predictions[f"week_{i}_inf"] = y_pred_test_quantile[i][:,0]

(2, 1352)


ValueError: Length of values (10) does not match length of index (1352)

### 4. Saving of the predictions

Saving of the predictions as a csv file

> The file must be named `predictions.csv`

In [ ]:
# save the predictions to a csv file
predictions["ObsDate"] = X_test.index
predictions.to_csv(f"{PREDS_DIR}predictions.csv", index=False)

Compression of the submission file.

> The file need to be compress for Codabench.

In [ ]:
# Create a ZIP file containing predictions.csv
with zipfile.ZipFile(f"{PREDS_DIR}predictions.zip", 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(f"{PREDS_DIR}predictions.csv", "predictions.csv")


You are ready to submit go to codabench and submit the zip file that have been generated in My Submissions > Phase 1.


You don't have to use this notebook to submit but the file file format must includes the following columns:
* station_code: Identification code of the station.
* ObsDate: Date of the prediction.
* for every week of prediction i from 0 to 3 :
    * week_i_pred
    * week_i_inf
    * week_i_sup


Save the dataset as a CSV file named predictions.csv. 
> The file must be named predictions.csv, but the .zip file can have any name.


Compress the CSV file into a .zip archive. 
> You cannot submit an uncompressed file. Ensure that the software you use does not create a subfolder inside the archive.


Submit your file in [Codabench](https://www.codabench.org/competitions/4335): 
> My Submissions > Phase 1 (keep all the tasks selected):

<img src="../images/submissions.png" alt="Experiment Diagram" style="width:75%;" style="text-align:center;" />
